論文<br>
https://arxiv.org/abs/2008.10010<br>
<br>
Github<br>
https://github.com/Rudrabha/Wav2Lip<br>
<br>
<a href="https://colab.research.google.com/github/kaz12tech/ai_demos/blob/master/6DRepNet_demo.ipynb\" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# ランタイムの設定
「ランタイム」→「ランタイムのタイプを変更」→「ハードウェアアクセラレータ」をCPUに変更

# 実行方法
「ランタイム」→「すべてのセルを実行」を選択

# GPU確認

In [ ]:
!nvidia-smi

# 環境セットアップ

## Githubからコードを取得

In [ ]:
%cd /content

!git clone https://github.com/zabique/Wav2Lip

# Commits on Aug 10, 2021使用
%cd /content/Wav2Lip
!git checkout b9759a3467cb1b7519f1a3b91f5a84cb4bc1ae4a

## ライブラリのインストール

In [ ]:
%cd /content/Wav2Lip

!pip install -r requirements.txt
!pip install moviepy==0.2.3.5 imageio==2.4.1
!pip install yt-dlp

## ライブラリのインポート

In [ ]:
%cd /content/Wav2Lip

from yt_dlp import YoutubeDL
from moviepy.video.fx.resize import resize
from moviepy.editor import VideoFileClip
from IPython.display import Audio, display

# 学習済みモデルのセットアップ

In [ ]:
%cd /content/Wav2Lip

#download the pretrained model
!wget -c https://iiitaphyd-my.sharepoint.com/personal/radrabha_m_research_iiit_ac_in/_layouts/15/download.aspx?share=EdjI7bZlgApMqsVoEUUXpLsBxqXbn5z8VTmoxp55YNDcIA \
      -O ./checkpoints/wav2lip_gan.pth
!wget -c https://iiitaphyd-my.sharepoint.com/:u:/g/personal/radrabha_m_research_iiit_ac_in/Eb3LEzbfuKlJiR600lQWRxgBIY27JZg80f7V9jtMfbNDaQ?e=TBFBVW \
      -O ./checkpoints/wav2lip.pth
!pip install https://raw.githubusercontent.com/AwaleSajil/ghc/master/ghc-1.0-py3-none-any.whl

#download pretrained model for face detection
!wget -c https://www.adrianbulat.com/downloads/python-fan/s3fd-619a316812.pth \
      -O ./face_detection/detection/sfd/s3fd.pth

# テスト動画&音声のセットアップ

## 入力動画の取得

In [ ]:
video_url = 'https://www.youtube.com/watch?v=-Xvpo0A5X6g' #@param {type:"string"}

#@markdown 動画の切り抜き範囲(秒)を指定してください。\
#@markdown 30秒以上の場合OOM発生の可能性が高いため注意
start_sec =  30#@param {type:"integer"}
end_sec =  35#@param {type:"integer"}

(start_pt, end_pt) = (start_sec, end_sec)

In [ ]:
!mkdir input_video

download_resolution = 720
full_video_path = './input_video/full_video.mp4'
input_clip_path = './input_video/clip_video.mp4'

# 動画ダウンロード
ydl_opts = {'format': f'best[height<={download_resolution}]', 'overwrites': True, 'outtmpl': full_video_path}
with YoutubeDL(ydl_opts) as ydl:
    ydl.download([video_url])

# 指定区間切り抜き
with VideoFileClip(full_video_path) as video:
    subclip = video.subclip(start_pt, end_pt)
    subclip.write_videofile(input_clip_path)

In [ ]:
# 動画の確認
clip = VideoFileClip(input_clip_path)
clip = resize(clip, height=420)
clip.ipython_display()

## 入力音声取得

In [ ]:
video_url = 'https://www.youtube.com/watch?v=fD-yyMzF8lI' #@param {type:"string"}

#@markdown 動画の切り抜き範囲(秒)を指定してください。\
#@markdown 30秒以上の場合OOM発生の可能性が高いため注意
start_sec =  63#@param {type:"integer"}
end_sec =  68#@param {type:"integer"}

(start_pt, end_pt) = (start_sec, end_sec)

In [ ]:
!mkdir input_audio

download_resolution = 720
full_video_path = './input_audio/full_video_en.mp4'
input_clip_path = './input_audio/clip_video_en.mp4'
input_audio_path = './input_audio/audio.mp3'

# 動画ダウンロード
ydl_opts = {'format': f'best[height<={download_resolution}]', 'overwrites': True, 'outtmpl': full_video_path}
with YoutubeDL(ydl_opts) as ydl:
    ydl.download([video_url])

# 指定区間切り抜き
with VideoFileClip(full_video_path) as video:
    subclip = video.subclip(start_pt, end_pt)
    subclip.write_videofile(input_clip_path)

# 音声抽出
videoclip = VideoFileClip(input_clip_path)
audioclip = videoclip.audio
audioclip.write_audiofile(input_audio_path)

In [ ]:
display(Audio(input_audio_path, autoplay=True))

# Wav2Lip

In [ ]:
%cd /content/Wav2Lip

!python inference.py \
  --checkpoint_path checkpoints/wav2lip_gan.pth \
  --face ./input_video/clip_video.mp4 \
  --audio ./input_audio/audio.mp3

In [ ]:
results = './results/result_voice.mp4'

# Wav2Lipの確認
clip = VideoFileClip(results)
clip = resize(clip, height=420)
clip.ipython_display()